In [1]:
from __future__ import print_function

import sys
from random import random
from operator import add

from pyspark.sql import SparkSession

In [2]:
import os
os.environ['PYSPARK_PYTHON'] = 'python3'

In [3]:
spark = SparkSession.builder\
        .appName("PySpark Pi")\
        .master("k8s://https://kubernetes.default")\
        .config("spark.executor.instances", 1)\
        .config("spark.driver.host", "10.44.3.8")\
        .config("spark.kubernetes.namespace", "jupiter")\
        .config("spark.kubernetes.container.image", "gcr.io/spark-operator/spark-py:v2.4.0")\
        .config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")\
        .config("spark.kubernetes.pyspark.pythonVersion", "3")\
        .getOrCreate()

In [4]:
def f(_):
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 <= 1 else 0

In [5]:
partitions = 2
n = 100000 * partitions
count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
print("Pi is roughly %f" % (4.0 * count / n))

Pi is roughly 3.139320
